In [ ]:
# data_cleaning_fe.py
"""
Cleans data, engineers features, and saves 'listings_clean' table & processed CSV for modeling.
Run: python data_cleaning_fe.py
"""
import sqlite3
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import os

DB = "resqfood.db"
OUT_CSV = "resqfood_processed.csv"

conn = sqlite3.connect(DB)
df = pd.read_sql("SELECT * FROM listings", conn, parse_dates=['time_posted'])
conn.close()

# --- Basic cleaning ---
# Drop duplicates
df.drop_duplicates(subset=['listing_id'], inplace=True)

# Fill missing (if any)
df['distance_km'].fillna(df['distance_km'].median(), inplace=True)
df['ambient_temp_c'].fillna(df['ambient_temp_c'].median(), inplace=True)
df['quantity'].fillna(df['quantity'].median(), inplace=True)

# Convert categorical to categories
for col in ['donor_type','food_type','unit','city','packaging','donor_reliability']:
    df[col] = df[col].astype('category')

# --- Feature engineering ---
# 1. Hour of posting and is_peak_hour
df['hour_posted'] = df['time_posted'].dt.hour
df['is_peak_hour'] = df['hour_posted'].apply(lambda x: 1 if 8 <= x <= 20 else 0)

# 2. Quantity per standard kg estimate (convert plates/boxes/pack to kg heuristically)
unit_to_kg = {'kg':1.0, 'plates':0.5, 'boxes':2.0, 'packs':1.0}
df['quantity_kg'] = df.apply(lambda r: r['quantity'] * unit_to_kg.get(r['unit'], 1.0), axis=1)

# 3. Create a simple donor reliability score
rel_map = {'Low':0.5, 'Medium':0.8, 'High':1.0}
df['donor_reliability_score'] = df['donor_reliability'].map(rel_map)

# 4. Time_since_cooked_hours
df['time_since_cooked_hours'] = df['time_since_cooked_mins'] / 60.0

# 5. Interaction features
df['qty_div_distance'] = df['quantity_kg'] / (df['distance_km'] + 0.5)
df['freshness_by_pack'] = df['freshness_score'] * df['donor_reliability_score']

# 6. One-hot encode categorical variables for modeling (but keep a copy of original)
categorical_cols = ['donor_type','food_type','city','packaging']
df_for_model = df.copy()
df_for_model = pd.get_dummies(df_for_model, columns=categorical_cols, drop_first=True)

# 7. Select features & targets
features = [
    'quantity_kg','distance_km','ambient_temp_c','time_since_cooked_hours',
    'donor_reliability_score','is_peak_hour','qty_div_distance'
] + [c for c in df_for_model.columns if c.startswith(('donor_type_','food_type_','city_','packaging_'))]

target_class = 'picked_up'
target_reg = 'freshness_score'

X = df_for_model[features]
y_class = df_for_model[target_class]
y_reg = df_for_model[target_reg]

# Save processed
processed = pd.concat([X, y_class, y_reg, df[['listing_id','time_posted']].reset_index(drop=True)], axis=1)
processed.to_csv(OUT_CSV, index=False)

# Write back to SQLite as a clean table
conn = sqlite3.connect(DB)
processed.to_sql("listings_clean", conn, if_exists="replace", index=False)
conn.close()

print("Saved processed CSV:", OUT_CSV)
